In [ ]:
# version 9
# 0. Reset output
!rm -rf /kaggle/working/*

# 1. Import và cấu hình¶
# ===============================
# 📦 IMPORT THƯ VIỆN CƠ BẢN
# ===============================
import os
import gc
import warnings

import numpy as np
import pandas as pd

# ===============================
# 📊 VISUALIZATION
# ===============================
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings("ignore")

# ===============================
# 🤖 MACHINE LEARNING - SKLEARN
# ===============================
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    StratifiedKFold
)

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    f1_score,
    classification_report,
    confusion_matrix
)

# ===============================
# ⚖️ IMBALANCED LEARNING
# ===============================
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 2. Hàm trích xuất đặc trưng (Core Engine)
from scipy.stats import skew

def extract_features_from_split(csv_path):
    if not os.path.exists(csv_path): return None
    df = pd.read_csv(csv_path)
    
    # 1. TÍNH SNR
    df['snr'] = df['Flux'] / (df['Flux_err'] + 1e-6)
    
    # 2. Thống kê
    aggs = df.groupby(['object_id', 'Filter']).agg({
        'Flux': ['max', 'min', 'mean', 'std', skew],
        'snr': ['max', 'mean']
    }).unstack()
    aggs.columns = [f'{col[0]}_{col[1]}_{col[2]}' for col in aggs.columns]
    
    # 3. Tính Màu (Quan trọng)
    if 'Flux_max_g' in aggs.columns and 'Flux_max_r' in aggs.columns:
        aggs['color_g_r'] = aggs['Flux_max_g'] - aggs['Flux_max_r']
    
    # 4. Biên độ
    filters = df['Filter'].unique()
    for f in filters:
        if f'Flux_max_{f}' in aggs.columns and f'Flux_min_{f}' in aggs.columns:
            aggs[f'amp_{f}'] = aggs[f'Flux_max_{f}'] - aggs[f'Flux_min_{f}']
            
    # 5. Số lượng quan sát
    counts = df.groupby('object_id').size().to_frame('n_obs')
    
    features = aggs.merge(counts, left_index=True, right_index=True)
    return features

# !!! BẠN NHỚ CHẠY LẠI HÀM LOAD DỮ LIỆU ĐỂ CẬP NHẬT full_train NHÉ !!!

# Hàm hỗ trợ load toàn bộ 20 splits
def load_all_splits(base_path, mode='train'):
    all_features = []
    print(f"Bắt đầu xử lý dữ liệu {mode} từ 20 splits...")
    
    for i in range(1, 21):
        split_name = f'split_{i:02d}' # Format 01, 02...
        file_name = f'{mode}_full_lightcurves.csv'
        full_path = os.path.join(base_path, split_name, file_name)
        
        print(f"Processing {split_name}...", end='\r')
        
        feats = extract_features_from_split(full_path)
        if feats is not None:
            all_features.append(feats)
            
        # Giải phóng bộ nhớ RAM
        del feats
        gc.collect()
        
    print(f"\nĐã xử lý xong {mode}!")
    # Gộp tất cả các split thành 1 DataFrame lớn
    return pd.concat(all_features)

# 3. Chạy xử lý dữ liệu (Mất khoảng 2-5 phút)
BASE_PATH = '/kaggle/input/mallorn-dataset'

# 1. Load Feature từ Lightcurves (Bước tốn thời gian nhất)
train_lc_features = load_all_splits(BASE_PATH, mode='train')
test_lc_features = load_all_splits(BASE_PATH, mode='test')

print("Kích thước Train features:", train_lc_features.shape)
print("Kích thước Test features:", test_lc_features.shape)

# 2. Load file Log (Metadata)
train_log = pd.read_csv(f'{BASE_PATH}/train_log.csv')
test_log = pd.read_csv(f'{BASE_PATH}/test_log.csv')

# 3. Gộp (Merge) Lightcurve Features vào Log Data
# Dùng left join để đảm bảo giữ nguyên thứ tự của file Log
full_train = train_log.merge(train_lc_features, on='object_id', how='left')
full_test = test_log.merge(test_lc_features, on='object_id', how='left')

# Điền 0 cho các giá trị NaN sinh ra do merge (ví dụ ngôi sao không có dữ liệu ở filter 'u')
full_train.fillna(0, inplace=True)
full_test.fillna(0, inplace=True)

display(full_train.head(3))

# 4. Chuẩn bị dữ liệu cho SVM (Scale & SMOTE)
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.calibration import CalibratedClassifierCV # <--- VŨ KHÍ MỚI
from sklearn.metrics import f1_score, classification_report
import numpy as np

# 1. Chuẩn bị dữ liệu
drop_cols = ['object_id', 'SpecType', 'English Translation', 'split', 'target', 'Z_err']
feature_cols = [c for c in full_train.columns if c not in drop_cols]

X = full_train[feature_cols]
y = full_train['target']
X_test_sub = full_test[feature_cols]

# Chia tập (Stratify)
X_train_org, X_val_org, y_train_org, y_val_org = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Huấn luyện SVM & Tinh chỉnh (Training & Tuning)
# Thay vì chạy 1 lần, chúng ta dùng Cross-Validation để đảm bảo kết quả ổn định.

# 2. Pipeline (Quay lại bản 0.3848)
# Lưu ý: Tôi bỏ class_weight='balanced' vì đã có SMOTE. 
# Dùng cả 2 đôi khi làm model bị nhiễu (Double balancing).
svm_pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),                 # Quay lại StandardScaler
    ('smote', SMOTE(random_state=42, k_neighbors=5)), 
    ('select', SelectKBest(score_func=f_classif)),
    ('svm', SVC(probability=True, kernel='rbf', random_state=42)) # Bỏ class_weight='balanced'
])

# 3. GridSearch
param_grid = {
    'select__k': [20, 30],
    'svm__C': [10, 50, 100],      # Tăng C lên để phạt lỗi nặng hơn
    'svm__gamma': ['scale', 0.1],
    'smote__sampling_strategy': [0.5, 0.75] # Tỷ lệ sinh vừa phải
}

print("Đang tìm Best SVM...")
grid = GridSearchCV(svm_pipeline, param_grid, cv=3, scoring='f1', verbose=1, n_jobs=-1)
grid.fit(X_train_org, y_train_org)

print("Best params:", grid.best_params_)
best_pipeline = grid.best_estimator_

# --- BƯỚC MỚI: HIỆU CHỈNH XÁC SUẤT (CALIBRATION) ---
# SVM thuần thường tự tin thái quá hoặc quá rụt rè. 
# CalibratedClassifierCV sẽ "nắn" lại xác suất cho chuẩn.
print("Đang hiệu chỉnh xác suất (Calibration)...")
calibrated_svm = CalibratedClassifierCV(best_pipeline, method='isotonic', cv='prefit')
calibrated_svm.fit(X_val_org, y_val_org) # Fit trên tập Val để học cách sửa lỗi

# Dự đoán thử trên Val
y_pred_val_calibrated = calibrated_svm.predict(X_val_org)
print("\nValidation F1 (Calibrated):", f1_score(y_val_org, y_pred_val_calibrated))

# Tinh chỉnh ngưỡng (Threshold Tuning)
from sklearn.metrics import precision_recall_curve

# Lấy xác suất từ model đã hiệu chỉnh
y_val_prob = calibrated_svm.predict_proba(X_val_org)[:, 1]

# Tìm ngưỡng tối ưu
precisions, recalls, thresholds = precision_recall_curve(y_val_org, y_val_prob)
f1_scores = np.divide(2 * precisions * recalls, precisions + recalls, out=np.zeros_like(precisions), where=(precisions + recalls)!=0)

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
print(f"Ngưỡng tối ưu: {best_threshold:.4f}")
print(f"Max F1 Val: {f1_scores[best_idx]:.4f}")

# 6. Dự đoán và Submit
# Dự đoán Test
y_test_prob = calibrated_svm.predict_proba(X_test_sub)[:, 1]
final_predictions = (y_test_prob >= best_threshold).astype(int)

submission = pd.DataFrame({
    'object_id': full_test['object_id'],
    'prediction': final_predictions
})

print(submission['prediction'].value_counts())
submission.to_csv('submission_svm_calibrated.csv', index=False)
print('Đã lưu file: submission_svm_calibrated.csv')


In [ ]:
# version 8
# 0. Reset output
!rm -rf /kaggle/working/*

# 1. Import và cấu hình¶
# ===============================
# 📦 IMPORT THƯ VIỆN CƠ BẢN
# ===============================
import os
import gc
import warnings

import numpy as np
import pandas as pd

# ===============================
# 📊 VISUALIZATION
# ===============================
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings("ignore")

# ===============================
# 🤖 MACHINE LEARNING - SKLEARN
# ===============================
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    StratifiedKFold
)

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    f1_score,
    classification_report,
    confusion_matrix
)

# ===============================
# ⚖️ IMBALANCED LEARNING
# ===============================
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 2. Hàm trích xuất đặc trưng (Core Engine)
from scipy.stats import skew, kurtosis

# Reset hàm feature tốt nhất
from scipy.stats import skew

def extract_features_from_split(csv_path):
    if not os.path.exists(csv_path): return None
    df = pd.read_csv(csv_path)
    
    # 1. TÍNH SNR (Tín hiệu trên nhiễu)
    df['snr'] = df['Flux'] / (df['Flux_err'] + 1e-6)
    
    # 2. Thống kê cơ bản
    aggs = df.groupby(['object_id', 'Filter']).agg({
        'Flux': ['max', 'min', 'mean', 'std', skew], # Skew rất quan trọng
        'snr': ['max', 'mean']
    }).unstack()
    aggs.columns = [f'{col[0]}_{col[1]}_{col[2]}' for col in aggs.columns]
    
    # 3. Tính Màu & Biên độ (Vật lý)
    if 'Flux_max_g' in aggs.columns and 'Flux_max_r' in aggs.columns:
        aggs['color_g_r'] = aggs['Flux_max_g'] - aggs['Flux_max_r']
        # Thêm tỷ lệ (Ratio) - SVM thích cái này hơn hiệu số
        aggs['ratio_g_r'] = aggs['Flux_max_g'] / (aggs['Flux_max_r'] + 1)

    filters = df['Filter'].unique()
    for f in filters:
        if f'Flux_max_{f}' in aggs.columns and f'Flux_min_{f}' in aggs.columns:
            aggs[f'amp_{f}'] = aggs[f'Flux_max_{f}'] - aggs[f'Flux_min_{f}']
            
    # 4. Số lượng quan sát
    counts = df.groupby('object_id').size().to_frame('n_obs')
    
    # Merge lại
    features = aggs.merge(counts, left_index=True, right_index=True)
    return features

# --- CHẠY LẠI BƯỚC LOAD DỮ LIỆU ---
# (Bạn nhớ chạy lại hàm load_all_splits và tạo full_train, full_test nhé)
# ...

# Hàm hỗ trợ load toàn bộ 20 splits
def load_all_splits(base_path, mode='train'):
    all_features = []
    print(f"Bắt đầu xử lý dữ liệu {mode} từ 20 splits...")
    
    for i in range(1, 21):
        split_name = f'split_{i:02d}' # Format 01, 02...
        file_name = f'{mode}_full_lightcurves.csv'
        full_path = os.path.join(base_path, split_name, file_name)
        
        print(f"Processing {split_name}...", end='\r')
        
        feats = extract_features_from_split(full_path)
        if feats is not None:
            all_features.append(feats)
            
        # Giải phóng bộ nhớ RAM
        del feats
        gc.collect()
        
    print(f"\nĐã xử lý xong {mode}!")
    # Gộp tất cả các split thành 1 DataFrame lớn
    return pd.concat(all_features)

# 3. Chạy xử lý dữ liệu (Mất khoảng 2-5 phút)
BASE_PATH = '/kaggle/input/mallorn-dataset'

# 1. Load Feature từ Lightcurves (Bước tốn thời gian nhất)
train_lc_features = load_all_splits(BASE_PATH, mode='train')
test_lc_features = load_all_splits(BASE_PATH, mode='test')

print("Kích thước Train features:", train_lc_features.shape)
print("Kích thước Test features:", test_lc_features.shape)

# 2. Load file Log (Metadata)
train_log = pd.read_csv(f'{BASE_PATH}/train_log.csv')
test_log = pd.read_csv(f'{BASE_PATH}/test_log.csv')

# 3. Gộp (Merge) Lightcurve Features vào Log Data
# Dùng left join để đảm bảo giữ nguyên thứ tự của file Log
full_train = train_log.merge(train_lc_features, on='object_id', how='left')
full_test = test_log.merge(test_lc_features, on='object_id', how='left')

# Điền 0 cho các giá trị NaN sinh ra do merge (ví dụ ngôi sao không có dữ liệu ở filter 'u')
full_train.fillna(0, inplace=True)
full_test.fillna(0, inplace=True)

display(full_train.head(3))

# 4. Chuẩn bị dữ liệu cho SVM (Scale & SMOTE)
# --- BƯỚC 4: CHUẨN BỊ DỮ LIỆU (SỬA LẠI) ---
# Chọn Feature
drop_cols = ['object_id', 'SpecType', 'English Translation', 'split', 'target', 'Z_err']
feature_cols = [c for c in full_train.columns if c not in drop_cols]
X = full_train[feature_cols]
y = full_train['target']
X_test_sub = full_test[feature_cols]

# Chia tập Train/Val GỐC (Chưa scale, chưa SMOTE)
# Stratify để đảm bảo tỷ lệ TDE ở 2 tập như nhau
X_train_org, X_val_org, y_train_org, y_val_org = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Số lượng TDE trong tập Val thực tế: {sum(y_val_org==1)}")

# 5. Huấn luyện SVM & Tinh chỉnh (Training & Tuning)
# changes for v8: Bagging + RobustScaler + RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import RobustScaler  # <--- THAY ĐỔI 1
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, classification_report, precision_recall_curve

# 1. CHUẨN BỊ DỮ LIỆU
# Lấy lại dữ liệu gốc từ full_train (đã chạy qua hàm extract_features chuẩn ở bước trước)
drop_cols = ['object_id', 'SpecType', 'English Translation', 'split', 'target', 'Z_err']
feature_cols = [c for c in full_train.columns if c not in drop_cols]

X = full_train[feature_cols]
y = full_train['target']
X_test_sub = full_test[feature_cols]

# Chia tập Train/Val (Stratify để giữ tỷ lệ TDE)
X_train_org, X_val_org, y_train_org, y_val_org = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. XÂY DỰNG PIPELINE "BAGGING SVM"
# SVM cơ sở (Base Estimator)
svc_base = SVC(
    kernel='rbf', 
    probability=True,       # Bắt buộc True để Bagging tính được xác suất trung bình
    class_weight='balanced', 
    random_state=42
)

svm_pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='median')),    # Dùng Median an toàn hơn Mean
    ('scaler', RobustScaler()),                       # Dùng RobustScaler chống nhiễu
    ('smote', SMOTE(random_state=42, k_neighbors=7)), # Tăng neighbor lên 7 cho mượt
    ('select', SelectKBest(score_func=f_classif)),    
    ('bagging', BaggingClassifier(                    # <--- THAY ĐỔI 2: Bọc trong Bagging
        estimator=svc_base,
        n_estimators=10,        # Tạo 10 model con (Ensemble)
        max_samples=0.8,        # Mỗi model học 80% dữ liệu
        bootstrap=True,
        random_state=42,
        n_jobs=-1               # Chạy song song
    )) 
])

# 3. TÌM THAM SỐ (GridSearch/RandomizedSearch)
# Lưu ý cú pháp: tên_bước__tên_tham_số
# Với Bagging: bagging__estimator__tham_số_của_svm
param_grid = {
    'select__k': [20, 30, 'all'],             # Số lượng feature giữ lại
    'bagging__estimator__C': [10, 50, 100],   # C của SVM con (tăng lên vì RobustScaler thu nhỏ data)
    'bagging__estimator__gamma': ['scale', 0.1],
    'smote__sampling_strategy': [0.5, 0.8]    # Tỷ lệ sinh mẫu
}

print("Đang chạy RandomizedSearchCV cho Bagging SVM...")
search = RandomizedSearchCV(
    svm_pipeline, 
    param_grid, 
    n_iter=10,    # Thử 10 tổ hợp tốt nhất
    cv=3, 
    scoring='f1', 
    verbose=2, 
    random_state=42,
    n_jobs=1      # Để 1 vì bên trong Bagging đã dùng đa nhân rồi
)

search.fit(X_train_org, y_train_org)
best_model = search.best_estimator_

print("\nBest params:", search.best_params_)

# Tinh chỉnh ngưỡng (Threshold Tuning)
# 4. TÌM NGƯỠNG TỐI ƯU (Threshold Tuning)
# Lấy xác suất dự đoán trên tập Val
y_val_prob = best_model.predict_proba(X_val_org)[:, 1]

# Tính đường cong P-R
precisions, recalls, thresholds = precision_recall_curve(y_val_org, y_val_prob)
f1_scores = np.divide(2 * precisions * recalls, precisions + recalls, out=np.zeros_like(precisions), where=(precisions + recalls)!=0)

# Chọn ngưỡng có F1 cao nhất
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
print(f"Ngưỡng tối ưu: {best_threshold:.4f}")
print(f"Validation F1 (Max): {f1_scores[best_idx]:.4f}")

# 6. Dự đoán và Submit
# 5. SUBMIT
# Dự đoán trên tập Test
y_test_prob = best_model.predict_proba(X_test_sub)[:, 1]
final_predictions = (y_test_prob >= best_threshold).astype(int)

submission = pd.DataFrame({
    'object_id': full_test['object_id'],
    'prediction': final_predictions
})

print("\nThống kê dự đoán:")
print(submission['prediction'].value_counts())

submission.to_csv('submission_svm_bagging_robust.csv', index=False)
print("Đã lưu file: submission_svm_bagging_robust.csv")


In [ ]:
# version 6
# 0. Reset output
!rm -rf /kaggle/working/*

# 1. Import và cấu hình
# ===============================
# 📦 IMPORT THƯ VIỆN CƠ BẢN
# ===============================
import os
import gc
import warnings

import numpy as np
import pandas as pd

# ===============================
# 📊 VISUALIZATION
# ===============================
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings("ignore")

# ===============================
# 🤖 MACHINE LEARNING - SKLEARN
# ===============================
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    StratifiedKFold
)

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    f1_score,
    classification_report,
    confusion_matrix
)

# ===============================
# ⚖️ IMBALANCED LEARNING
# ===============================
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 2. Hàm trích xuất đặc trưng (Core Engine)
from scipy.stats import skew, kurtosis

# Reset hàm feature tốt nhất
from scipy.stats import skew

def extract_features_from_split(csv_path):
    if not os.path.exists(csv_path): return None
    df = pd.read_csv(csv_path)
    
    # 1. TÍNH SNR (Tín hiệu trên nhiễu)
    df['snr'] = df['Flux'] / (df['Flux_err'] + 1e-6)
    
    # 2. Thống kê cơ bản
    aggs = df.groupby(['object_id', 'Filter']).agg({
        'Flux': ['max', 'min', 'mean', 'std', skew], # Skew rất quan trọng
        'snr': ['max', 'mean']
    }).unstack()
    aggs.columns = [f'{col[0]}_{col[1]}_{col[2]}' for col in aggs.columns]
    
    # 3. Tính Màu & Biên độ (Vật lý)
    if 'Flux_max_g' in aggs.columns and 'Flux_max_r' in aggs.columns:
        aggs['color_g_r'] = aggs['Flux_max_g'] - aggs['Flux_max_r']
        # Thêm tỷ lệ (Ratio) - SVM thích cái này hơn hiệu số
        aggs['ratio_g_r'] = aggs['Flux_max_g'] / (aggs['Flux_max_r'] + 1)

    filters = df['Filter'].unique()
    for f in filters:
        if f'Flux_max_{f}' in aggs.columns and f'Flux_min_{f}' in aggs.columns:
            aggs[f'amp_{f}'] = aggs[f'Flux_max_{f}'] - aggs[f'Flux_min_{f}']
            
    # 4. Số lượng quan sát
    counts = df.groupby('object_id').size().to_frame('n_obs')
    
    # Merge lại
    features = aggs.merge(counts, left_index=True, right_index=True)
    return features

# Hàm hỗ trợ load toàn bộ 20 splits
def load_all_splits(base_path, mode='train'):
    all_features = []
    print(f"Bắt đầu xử lý dữ liệu {mode} từ 20 splits...")
    
    for i in range(1, 21):
        split_name = f'split_{i:02d}' # Format 01, 02...
        file_name = f'{mode}_full_lightcurves.csv'
        full_path = os.path.join(base_path, split_name, file_name)
        
        print(f"Processing {split_name}...", end='\r')
        
        feats = extract_features_from_split(full_path)
        if feats is not None:
            all_features.append(feats)
            
        # Giải phóng bộ nhớ RAM
        del feats
        gc.collect()
        
    print(f"\nĐã xử lý xong {mode}!")
    # Gộp tất cả các split thành 1 DataFrame lớn
    return pd.concat(all_features)

# 3. Chạy xử lý dữ liệu (Mất khoảng 2-5 phút)
BASE_PATH = '/kaggle/input/mallorn-dataset'

# 1. Load Feature từ Lightcurves (Bước tốn thời gian nhất)
train_lc_features = load_all_splits(BASE_PATH, mode='train')
test_lc_features = load_all_splits(BASE_PATH, mode='test')

print("Kích thước Train features:", train_lc_features.shape)
print("Kích thước Test features:", test_lc_features.shape)

# 2. Load file Log (Metadata)
train_log = pd.read_csv(f'{BASE_PATH}/train_log.csv')
test_log = pd.read_csv(f'{BASE_PATH}/test_log.csv')

# 3. Gộp (Merge) Lightcurve Features vào Log Data
# Dùng left join để đảm bảo giữ nguyên thứ tự của file Log
full_train = train_log.merge(train_lc_features, on='object_id', how='left')
full_test = test_log.merge(test_lc_features, on='object_id', how='left')

# Điền 0 cho các giá trị NaN sinh ra do merge (ví dụ ngôi sao không có dữ liệu ở filter 'u')
full_train.fillna(0, inplace=True)
full_test.fillna(0, inplace=True)

display(full_train.head(3))

# 4. Chuẩn bị dữ liệu cho SVM (Scale & SMOTE)
# --- BƯỚC 4: CHUẨN BỊ DỮ LIỆU (SỬA LẠI) ---
# Chọn Feature
drop_cols = ['object_id', 'SpecType', 'English Translation', 'split', 'target', 'Z_err']
feature_cols = [c for c in full_train.columns if c not in drop_cols]
X = full_train[feature_cols]
y = full_train['target']
X_test_sub = full_test[feature_cols]

# Chia tập Train/Val GỐC (Chưa scale, chưa SMOTE)
# Stratify để đảm bảo tỷ lệ TDE ở 2 tập như nhau
X_train_org, X_val_org, y_train_org, y_val_org = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Số lượng TDE trong tập Val thực tế: {sum(y_val_org==1)}")

# 5. Huấn luyện SVM & Tinh chỉnh (Training & Tuning)
svm_pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='mean')),      # 1. Điền giá trị thiếu
    ('scaler', StandardScaler()),                     # 2. Chuẩn hóa
    ('smote', SMOTE(random_state=42, k_neighbors=5)), # 3. Sinh dữ liệu (chỉ trên train)
    ('select', SelectKBest(score_func=f_classif)),    # 4. Chọn đặc trưng tốt nhất
    ('svm', SVC(probability=True, kernel='rbf', class_weight='balanced', random_state=42)) # 5. SVM
])

# Thiết lập tham số GridSearch
param_grid = {
    'select__k': [20, 30, 'all'],           # Giữ lại 20, 30 hoặc tất cả feature
    'svm__C': [1, 10, 100],                 # Tham số C của SVM
    'svm__gamma': ['scale', 0.1],           # Tham số Gamma
    'smote__sampling_strategy': [0.5, 1.0]  # Tỷ lệ sinh dữ liệu
}

print("Đang chạy GridSearch cho SVM (Quy trình chuẩn)...")
# cv=3: Chia 3 fold kiểm tra chéo
grid = GridSearchCV(svm_pipeline, param_grid, cv=3, scoring='f1', verbose=2, n_jobs=-1)

# Lưu ý: Dùng X_train_org và y_train_org (Dữ liệu GỐC chưa qua xử lý)
grid.fit(X_train_org, y_train_org)

print("\nBest params:", grid.best_params_)
best_model = grid.best_estimator_

# --- ĐÁNH GIÁ TRUNG THỰC ---
# Dự đoán trên tập Val gốc (Pipeline sẽ tự scale, bạn chỉ cần đưa dữ liệu thô vào)
y_pred_val = best_model.predict(X_val_org)
print("\nValidation F1 Score (Real):", f1_score(y_val_org, y_pred_val))
print(classification_report(y_val_org, y_pred_val))

# Tinh chỉnh lần 2 (thu nhỏ grid)
svm_pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42, k_neighbors=5)), 
    ('select', SelectKBest(score_func=f_classif, k=20)), # CHỈ GIỮ LẠI 20 FEATURE TỐT NHẤT
    ('svm', SVC(probability=True, kernel='rbf', class_weight='balanced', random_state=42))
])

param_grid = {
    'select__k': [15, 25, 'all'],     # Thử chọn 15, 25 hoặc lấy hết feature
    'svm__C': [1, 10, 50],            # C càng lớn càng phạt lỗi mạnh (dễ overfit)
    'svm__gamma': ['scale', 0.01, 0.1],
    'smote__sampling_strategy': [0.5, 0.8, 1.0]
}

print("Đang chạy GridSearch cho SVM (Quy trình chuẩn)...")
grid = GridSearchCV(svm_pipeline, param_grid, cv=3, scoring='f1', verbose=2, n_jobs=-1)
grid.fit(X_train_org, y_train_org)

print("\nBest params:", grid.best_params_)
best_model = grid.best_estimator_

# --- ĐÁNH GIÁ TRUNG THỰC ---
y_pred_val = best_model.predict(X_val_org)
print("\nValidation F1 Score (Real):", f1_score(y_val_org, y_pred_val))
print(classification_report(y_val_org, y_pred_val))

# Tinh chỉnh ngưỡng (Threshold Tuning)
from sklearn.metrics import precision_recall_curve

# --- 1. TÌM NGƯỠNG TỐI ƯU (Threshold Tuning) ---
# Dự đoán xác suất trên tập Validation
y_val_prob = best_model.predict_proba(X_val_org)[:, 1]

# Tính Precision-Recall curve
precisions, recalls, thresholds = precision_recall_curve(y_val_org, y_val_prob)

# Tính F1 score cho từng ngưỡng (Xử lý chia cho 0 để tránh lỗi)
f1_scores = np.divide(2 * precisions * recalls, precisions + recalls, out=np.zeros_like(precisions), where=(precisions + recalls)!=0)

# Lấy ngưỡng có F1 cao nhất
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

print(f"Ngưỡng tối ưu (Best Threshold): {best_threshold:.4f}")
print(f"F1 Score kỳ vọng tại ngưỡng này: {best_f1:.4f}")

# 6. Dự đoán và Submit
# --- 2. DỰ ĐOÁN VÀ SUBMIT ---

# Dự đoán xác suất trên tập Test thật
# LƯU Ý QUAN TRỌNG: Dùng X_test_sub (dữ liệu thô), KHÔNG dùng X_test_scaled
y_test_prob = best_model.predict_proba(X_test_sub)[:, 1]

# Áp dụng ngưỡng tối ưu
final_predictions = (y_test_prob >= best_threshold).astype(int)

# Tạo file submission
submission = pd.DataFrame({
    'object_id': full_test['object_id'],
    'prediction': final_predictions
})

# Kiểm tra phân phối dự đoán
print("\nThống kê dự đoán (Test Set):")
print(submission['prediction'].value_counts())

submission.to_csv('submission_svm_final.csv', index=False)
print("Đã lưu file: submission_svm_final.csv")
